In [23]:
import sys
sys.path.append('..')
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sympy import simplify_logic
import time
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import _tree, export_text
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold

from deep_logic.utils.base import validate_network, set_seed, tree_to_formula
from deep_logic.utils.relu_nn import get_reduced_model, prune_features
from deep_logic.utils.psi_nn import prune_equal_fanin
from deep_logic import logic

results_dir = 'results/omalizumab'
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
seed = 42
n_rep = 10
tot_epochs = 6001
prune_epochs = 3001

In [13]:
gene_expression_matrix = pd.read_csv('data/omalizumab/reduced_w_1/data.csv', index_col=None, header=None)
labels = pd.read_csv('data/omalizumab/reduced_w_1/tempLabels_W-1.csv', index_col=None, header=None)
genes = pd.read_csv('data/omalizumab/reduced_w_1/features.csv', index_col=None, header=None)
gene_expression_matrix

,0,1,2,3,4
0,3.320000,3.320000,3.32000,6.941536,6.590419
1,4.232978,3.320000,3.32000,7.279548,6.476784
2,3.320000,4.200609,3.32000,7.741600,4.643134
3,3.320000,3.320000,3.32000,7.276600,5.953452
4,3.320000,3.320000,3.32000,7.224628,6.555227
...,...,...,...,...,...
56,3.320000,3.320000,3.32000,7.660182,6.128603
57,3.320000,3.700430,3.45131,7.809826,6.153968
58,3.320000,3.320000,3.32000,7.580588,6.134398
59,4.174319,3.320000,3.32000,7.016004,7.124143


In [30]:
encoder = LabelEncoder()
labels_encoded = encoder.fit_transform(labels.values)
labels_encoded_noncontrols = labels_encoded[labels_encoded!=0] - 1

data_controls = gene_expression_matrix[labels_encoded==0]
data = gene_expression_matrix[labels_encoded!=0]

gene_signature = data_controls.mean(axis=0)
data_scaled = data - gene_signature

scaler = MinMaxScaler((0, 1))
scaler.fit(data_scaled)
data_normalized = scaler.transform(data_scaled)

x = torch.FloatTensor(data_normalized)
y = torch.FloatTensor(labels_encoded_noncontrols)
print(x.shape)
print(y.shape)

torch.Size([40, 5])
torch.Size([40])


c:\users\pietr\anaconda3\envs\deep-logic\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [31]:
concepts = list(genes.values.squeeze())
concepts

['ILMN_3286286',
 'ILMN_1775520',
 'ILMN_1656849',
 'ILMN_1781198',
 'ILMN_1665457']

In [94]:
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)

In [87]:
def train_nn(x_train, y_train, need_pruning, seed, device, relu=False, verbose=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    layers = [
        torch.nn.Linear(x_train.size(1), 10),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(10, 3),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(3, 1),
        torch.nn.Sigmoid(),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    loss_form = torch.nn.BCELoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train).squeeze()
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.001 * torch.norm(module.weight, 1)
                loss += 0.001 * torch.norm(module.bias, 1)
                break

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning:
            prune_features(model, n_classes=1, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0 and verbose:
            y_pred_d = y_pred > 0.5
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [77]:
def train_psi_nn(x_train, y_train, need_pruning, seed, device, verbose=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device).to(torch.float)
    layers = [
        torch.nn.Linear(x_train.size(1), 10),
        torch.nn.Sigmoid(),
        torch.nn.Linear(10, 4),
        torch.nn.Sigmoid(),
        torch.nn.Linear(4, 1),
        torch.nn.Sigmoid(),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_form = torch.nn.BCELoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train).squeeze()
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.00001 * torch.norm(module.weight, 1)

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning:
            model = prune_equal_fanin(model, 2, validate=True, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0 and verbose:
            y_pred_d = y_pred > 0.5
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [78]:
def c_to_y(method, need_pruning, relu, verbose=False):
    methods = []
    splits = []
    explanations = []
    explanations_inv = []
    model_accuracies = []
    explanation_accuracies = []
    explanation_accuracies_inv = []
    elapsed_times = []
    elapsed_times_inv = []
    for split, (train_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(), y.cpu().detach().numpy())):
        print(f'Split [{split+1}/{n_splits}]')
        x_train, x_test = torch.FloatTensor(x[train_index]), torch.FloatTensor(x[test_index])
        y_train, y_test = torch.FloatTensor(y[train_index]), torch.FloatTensor(y[test_index])
    
        explanation, explanation_inv = '', ''
        explanation_accuracy, explanation_accuracy_inv = 0, 0
        
        if method == 'tree':
            classifier = DecisionTreeClassifier(random_state=seed)
            classifier.fit(x_train.detach().numpy(), y_train.detach().numpy())
            y_preds = classifier.predict(x_test.detach().numpy())
            model_accuracy = accuracy_score(y_test.detach().numpy(), y_preds)

            target_class = 1
            start = time.time()
            explanation = tree_to_formula(classifier, concepts, target_class)
            elapsed_time = time.time() - start

            target_class_inv = 0
            start = time.time()
            explanation_inv = tree_to_formula(classifier, concepts, target_class_inv)
            elapsed_time = time.time() - start
        
        else:
            if method == 'psi':
                # positive class
                target_class = 1
                model = train_psi_nn(x_train, y_train.eq(target_class), need_pruning, seed, device, verbose)
                y_preds = model(x_test.to(device)).cpu().detach().numpy()
                model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds>0.5)
                
            else:
                model = train_nn(x_train, y_train, need_pruning, seed, device, relu, verbose)
                y_preds = model(x_test.to(device)).cpu().detach().numpy()
                model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds>0.5)

            # positive class
            target_class = 1
            start = time.time()
            if method == 'psi':
                global_explanation = logic.generate_fol_explanations(model, device)[0]
            else:
                global_explanation, _, _ = logic.relu_nn.combine_local_explanations(model, 
                                                                                   x_train.to(device), y_train.to(device), 
                                                                                   target_class=target_class,
                                                                                   method=method, device=device)
            elapsed_time = time.time() - start
            
            if global_explanation:
                explanation_accuracy, _ = logic.base.test_explanation(global_explanation, target_class, x_test, y_test)
                explanation = logic.base.replace_names(global_explanation, concepts)

            # negative class
            target_class_inv = 0
            if method == 'psi':
                model = train_psi_nn(x_train, y_train.eq(target_class_inv), need_pruning, seed, device, verbose)
            
            start = time.time()
            if method == 'psi':
                global_explanation_inv = logic.generate_fol_explanations(model, device)[0]
            else:
                global_explanation_inv, _, _ = logic.relu_nn.combine_local_explanations(model, 
                                                                                       x_train.to(device), 
                                                                                       y_train.to(device), 
                                                                                       target_class=target_class_inv,
                                                                                       method=method, device=device)
            elapsed_time_inv = time.time() - start
            if global_explanation_inv:
                explanation_accuracy_inv, _ = logic.base.test_explanation(global_explanation_inv, 
                                                                          target_class_inv, x_test, y_test)
                explanation_inv = logic.base.replace_names(global_explanation_inv, concepts)
        
        if verbose:
            print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
            print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
            print(f'\t Elapsed time {elapsed_time}')
            print(f'\t Class {target_class_inv} - Global explanation: "{explanation_inv}" - Accuracy: {explanation_accuracy_inv:.4f}')
            print(f'\t Elapsed time {elapsed_time_inv}')

        methods.append(method)
        splits.append(split)
        explanations.append(explanation)
        explanations_inv.append(explanation_inv)
        model_accuracies.append(model_accuracy)
        explanation_accuracies.append(explanation_accuracy)
        explanation_accuracies_inv.append(explanation_accuracy_inv)
        elapsed_times.append(elapsed_time)
        elapsed_times_inv.append(elapsed_time_inv)
    
    results = pd.DataFrame({
        'method': methods,
        'split': splits,
        'explanation': explanations,
        'explanation_inv': explanations_inv,
        'model_accuracy': model_accuracies,
        'explanation_accuracy': explanation_accuracies,
        'explanation_accuracy_inv': explanation_accuracies_inv,
        'elapsed_time': elapsed_times,
        'elapsed_time_inv': elapsed_times_inv,
    })
    results.to_csv(os.path.join(results_dir, f'results_{method}.csv'))
    
    return results

In [93]:
model = train_nn(x, y, need_pruning=True, seed=seed, device=device, relu=False, verbose=True)
y_preds = model(x.to(device)).cpu().detach().numpy()
model_accuracy = accuracy_score(y.cpu().detach().numpy(), y>0.5)

# positive class
target_class = 1
start = time.time()
global_explanation, _, _ = logic.relu_nn.combine_local_explanations(model, 
                                                                    x.to(device), y.to(device), 
                                                                    target_class=target_class,
                                                                    topk_explanations=10,
                                                                    method='pruning', device=device)
elapsed_time = time.time() - start

if global_explanation:
    explanation_accuracy, _ = logic.base.test_explanation(global_explanation, target_class, x, y)
    explanation = logic.base.replace_names(global_explanation, concepts)

print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
print(f'\t Elapsed time {elapsed_time}')

	 Epoch 0: train accuracy: 0.7500
	 Epoch 500: train accuracy: 1.0000
	 Epoch 1000: train accuracy: 1.0000
	 Epoch 1500: train accuracy: 1.0000
	 Epoch 2000: train accuracy: 1.0000
	 Epoch 2500: train accuracy: 1.0000
	 Epoch 3000: train accuracy: 1.0000
	 Epoch 3500: train accuracy: 1.0000
	 Epoch 4000: train accuracy: 1.0000
	 Epoch 4500: train accuracy: 1.0000
	 Epoch 5000: train accuracy: 1.0000
	 Epoch 5500: train accuracy: 1.0000
	 Epoch 6000: train accuracy: 1.0000
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "(~ILMN_3286286 & ~ILMN_1775520 & ~ILMN_1656849) | (~ILMN_3286286 & ~ILMN_1656849 & ~ILMN_1781198) | (~ILMN_1775520 & ~ILMN_1656849 & ~ILMN_1781198)" - Accuracy: 97.5000
	 Elapsed time 0.04890179634094238


# General pruning

In [95]:
method = 'pruning'
need_pruning = True
relu = False
results_pruning = c_to_y(method, need_pruning, relu, verbose=False)
results_pruning

Split [1/10]
Split [2/10]
Split [3/10]
Split [4/10]
Split [5/10]
Split [6/10]
Split [7/10]
Split [8/10]
Split [9/10]
Split [10/10]


,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,pruning,0,~ILMN_3286286 & ~ILMN_1656849 & ~ILMN_1781198,(ILMN_1656849 & ILMN_1781198 & ILMN_1665457 & ...,1.0,50.0,75.0,0.055851,0.048870
1,pruning,1,~ILMN_3286286 & ~ILMN_1656849 & ~ILMN_1781198,(ILMN_3286286 & ILMN_1781198 & ~ILMN_1656849 &...,1.0,50.0,75.0,0.052821,0.042885
2,pruning,2,~ILMN_3286286 & ~ILMN_1775520 & ~ILMN_1656849,(ILMN_1775520 & ILMN_1781198 & ~ILMN_3286286 &...,1.0,100.0,75.0,0.048869,0.064827
3,pruning,3,~ILMN_3286286 & ~ILMN_1656849 & ~ILMN_1781198,(ILMN_3286286 & ILMN_1781198 & ~ILMN_1656849 &...,1.0,50.0,75.0,0.059353,0.053855
4,pruning,4,~ILMN_3286286 & ~ILMN_1656849 & ~ILMN_1781198,(ILMN_3286286 & ILMN_1781198 & ~ILMN_1656849 &...,1.0,50.0,100.0,0.051861,0.081782
5,pruning,5,~ILMN_3286286 & ~ILMN_1775520 & ~ILMN_1656849 ...,ILMN_1656849 & ILMN_1781198 & ~ILMN_3286286 & ...,1.0,25.0,75.0,0.070779,0.072300
6,pruning,6,~ILMN_3286286 & ~ILMN_1775520 & ~ILMN_1656849,(ILMN_1775520 & ILMN_1781198 & ~ILMN_3286286 &...,1.0,100.0,75.0,0.083778,0.070807
7,pruning,7,~ILMN_3286286 & ~ILMN_1656849 & ~ILMN_1781198,(ILMN_3286286 & ILMN_1781198 & ~ILMN_1656849 &...,1.0,50.0,75.0,0.063799,0.051859
8,pruning,8,~ILMN_3286286 & ~ILMN_1775520 & ~ILMN_1656849,(ILMN_1775520 & ILMN_1781198 & ~ILMN_3286286 &...,1.0,100.0,75.0,0.051926,0.054788
9,pruning,9,~ILMN_3286286 & ~ILMN_1775520 & ~ILMN_1656849 ...,ILMN_1656849 & ILMN_1781198 & ~ILMN_3286286 & ...,1.0,75.0,75.0,0.065828,0.075335


# LIME

In [ ]:
method = 'lime'
need_pruning = False
relu = False
results_lime = c_to_y(method, need_pruning, relu, False)
results_lime

# ReLU

In [ ]:
method = 'weights'
need_pruning = False
relu = True
results_weights = c_to_y(method, need_pruning, relu, False)
results_weights

# Psi network

In [ ]:
method = 'psi'
need_pruning = True
relu = False
results_psi = c_to_y(method, need_pruning, relu, verbose=False)
results_psi

# Decision tree

In [ ]:
method = 'tree'
need_pruning = False
relu = False
results_tree = c_to_y(method, need_pruning, relu, verbose=False)
results_tree

# Summary

In [43]:
cols = ['model_accuracy', 'explanation_accuracy', 'explanation_accuracy_inv', 'elapsed_time', 'elapsed_time_inv']
mean_cols = [f'{c}_mean' for c in cols]
sem_cols = [f'{c}_sem' for c in cols]

# pruning
df_mean = results_pruning[cols].mean()
df_sem = results_pruning[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_pruning = pd.concat([df_mean, df_sem])
summary_pruning.name = 'pruning'

# lime
df_mean = results_lime[cols].mean()
df_sem = results_lime[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_lime = pd.concat([df_mean, df_sem])
summary_lime.name = 'lime'

# weights
df_mean = results_weights[cols].mean()
df_sem = results_weights[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_weights = pd.concat([df_mean, df_sem])
summary_weights.name = 'weights'

# psi
df_mean = results_psi[cols].mean()
df_sem = results_psi[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_psi = pd.concat([df_mean, df_sem])
summary_psi.name = 'psi'

# tree
df_mean = results_tree[cols].mean()
df_sem = results_tree[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_tree = pd.concat([df_mean, df_sem])
summary_tree.name = 'tree'

summary = pd.concat([summary_pruning, 
                     summary_lime, 
                     summary_weights, 
                     summary_psi, 
                     summary_tree], axis=1).T
summary.columns = mean_cols + sem_cols
summary

,model_accuracy_mean,explanation_accuracy_mean,explanation_accuracy_inv_mean,elapsed_time_mean,elapsed_time_inv_mean,model_accuracy_sem,explanation_accuracy_sem,explanation_accuracy_inv_sem,elapsed_time_sem,elapsed_time_inv_sem
pruning,0.975,0.675,0.800,0.068861,0.041138,0.025000,0.098953,0.050000,0.004212,0.003383
lime,0.850,1.000,0.250,24.077748,10.371542,0.040825,0.000000,0.000000,0.115518,0.075365
weights,1.000,0.750,0.775,0.109159,0.050979,0.000000,0.091287,0.078617,0.003653,0.003413
psi,0.800,0.900,0.800,0.035007,0.029819,0.033333,0.040825,0.033333,0.002218,0.003012
tree,0.875,0.875,0.875,0.000000,0.000000,0.055902,0.055902,0.055902,0.000000,0.000000


In [44]:
summary.to_csv(os.path.join(results_dir, 'summary.csv'))